# Process Handbook

This notebook processes the example handbook (CAHRC_HR_Manual.txt).  This is done in a simple fashion using the following heuristic: If a line of text consisting of less than 5 words is followed by paragraphs of text the assume the line of text with less than 5 words is a topic (i.e. the topic of a question an employee might ask) and that the paragraphs of text are the answer to that question (called action_text for the lack of a better term).

When a topic and action_text are found these are stored in Cloud Datastore as a key-value pair with the topic as the key and the action_text as the value.

In [1]:
!pip uninstall -y google-cloud-datastore

Skipping google-cloud-datastore as it is not installed.


In [2]:
!pip install google-cloud-datastore

    100% |████████████████████████████████| 133kB 4.1MB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 11.9MB/s ta 0:00:01
    100% |████████████████████████████████| 102kB 9.2MB/s a 0:00:011
    100% |████████████████████████████████| 1.3MB 8.1MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 18.2MB/s ta 0:00:01
    100% |████████████████████████████████| 20.0MB 1.2MB/s eta 0:00:01
  Running setup.py bdist_wheel for grpcio ... done
  Stored in directory: /content/.cache/pip/wheels/e4/2a/04/8be8b1db08cb3def3c8f6e608c9fcdc1509ce7587856793b3f
Successfully built grpcio
pandas-gbq 0.3.0 has requirement google-cloud-bigquery>=0.28.0, but you'll have google-cloud-bigquery 0.23.0 which is incompatible.
google-cloud-monitoring 0.28.0 has requirement google-api-core<0.2.0dev,>=0.1.1, but you'll have google-api-core 1.22.1 which is incompatible.
google-cloud-monitoring 0.28.0 has requirement google-cloud-core<0.29dev,>=0.28.0, but you'll have google-cloud-core 1.4.

Hit Reset Session > Restart, then resume with the following cells. 

In [1]:
from google.cloud import datastore

In [2]:
datastore_client = datastore.Client()

In [3]:
employee_handbook = open('CAHRC_HR_Manual.txt', 'r')
while True:
  
  topic = employee_handbook.readline()
  if not(topic):
    break
  
  if (topic != '\r\n') and (len(topic.split(' ')) < 5):
  
    action_text = ''
        
    last_line = ''
    line = employee_handbook.readline()
    
    while (last_line != '\r\n') and (line != '\r\n') and (len(line.split(' ')) > 5):
      
      action_text += line
      last_line = line
      line = employee_handbook.readline()
      
    if action_text != '':
      
      kind = 'Topic'
      topic_key = datastore_client.key(kind, topic.strip().lower())

      topic = datastore.Entity(key=topic_key)
      topic['action_text'] = action_text

      datastore_client.put(topic)

      print('Saved {}: {}'.format(topic.key.name, topic['action_text']))

Saved employment equity: [THE ORGANIZATION] is an equal opportunity employer and employs personnel without regard to race, ancestry, place of origin, colour, ethnic origin, language, citizenship, creed, religion, gender, sexual orientation, age, marital status, physical and/or mental handicap or financial ability. While remaining alert and sensitive to the issue of fair and equitable treatment for all, [THE ORGANIZATION] has a special concern with the participation and advancement of members of four designated groups that have traditionally been disadvantaged in employment: women, visible minorities, aboriginal peoples and persons with disabilities.

Saved recruitment and selection: All employment opportunities at [the organization]  are posted for a minimum 10 working day period.  They are posted on [THE ORGANIZATION]’s website and on the websites of affiliated organizations.  Occasionally, they are posted on employment websites or with an employment agency.  Applications are encourag

Saved termination without cause: An Employment Contract may be terminated by the Employer at any time and for any reason on a without cause basis, upon the provision of notice or payment of notice instead , and severance pay if applicable, as is minimally required by the ESA, as amended from time to time.  In addition to notice, and pursuant to the ESA, the employee shall be entitled to an additional one (1) week’s notice or payment in lieu of notice for every year of completed service (severance pay) with the Employer to a maximum of sixteen (16) weeks’ notice inclusive of the notice requirements under the ESA. The notice as described in this paragraph is inclusive of all statutory and common law entitlements to notice or payment in lieu of notice.  Upon satisfaction of the requirements under this paragraph, the Employer shall have satisfied any and all obligations to the employee, whether under the ESA, as amended, or at common law.  The notice requirement contained in this clause co